In [1]:
import os
import math
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow.keras as keras
import tensorflow as tf
import pickle

In [2]:
# Save the MFCC and export them as numpy arrays
def save_mfcc(dataset_path,
              track_duration=30,
              sr=22050,
              n_mfcc=13,
              n_fft=2048,
              hop_length=512,
              num_segments=6):

    mfcc_data = []
    labels = []

    # Map labels to genre
    label_dict = {
        'jazz': 0,
        'reggae': 1,
        'rock': 2,
        'blues': 3,
        'hiphop': 4,
        'country': 5,
        'metal': 6,
        'classical': 7,
        'disco': 8,
        'pop': 9
    }

    samples_per_track = sr * track_duration
    num_samples_per_segment = int(samples_per_track / num_segments)
    expected_num_mfcc_per_segment = math.ceil(num_samples_per_segment / hop_length)

    # Iterate through the files
   
    for file in os.scandir(dataset_path):

        # Parse the file names by genre
        label_key = file.name.split('.')[0]

        # Store the integer label based on genre
        label = label_dict[label_key]

        # Process the files
        signal, sr = librosa.load(file, sr=sr)

        # Progress the segments and extract the MFCC
        for s in range(num_segments):
            start_sample = num_samples_per_segment * s
            finish_sample = start_sample + num_samples_per_segment

            # Ensure MFCC has correct vector length, if it does save it
            mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                        sr=sr,
                                        n_fft=n_fft,
                                        n_mfcc=n_mfcc,
                                        hop_length=hop_length)
            mfcc = mfcc.T

            if len(mfcc) == expected_num_mfcc_per_segment:
                mfcc_data.append(mfcc.tolist())
                labels.append(label)

    mfcc_data = np.array(mfcc_data)
    labels = np.array(labels)

    return mfcc_data, labels

In [13]:
mfcc, labels = save_mfcc('../../audio_data/')

In [14]:
print(f'mfcc.shape: {mfcc.shape}')

mfcc.shape: (5992, 216, 13)


In [15]:
print(f'labels.shape: {labels.shape}')

labels.shape: (5992,)


In [17]:
print("Saving the data")
np.savez('../../audio/audio_data_MFCC/audio_output.npz', mfcc=mfcc, labels=labels)

Saving the data
